In [31]:
import time
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
with open('data/swannsway.txt', 'r') as f:
    swann = f.read()
len(swann)

1103301

In [3]:
letters = sorted(list(set(swann)))
" ".join(letters)

'\n   ! " $ % \' ( ) * , - . / 0 1 2 3 4 5 6 7 8 9 : ; ? @ A B C D E F G H I J K L M N O P Q R S T U V W X Y Z _ a b c d e f g h i j k l m n o p q r s t u v w x y z ° Î Ï à â ç è é ê ë î ï ô'

In [23]:
letter2int = {letters[i]:i for i in range(len(letters))}
int2letters = {i:letters[i] for i in range(len(letters))}
swann_encoded = np.array([letter2int[letter] for letter in swann], dtype=np.int32)

In [24]:
len(swann_encoded)

1103301

In [25]:
def get_batches(arr, n_seqs, n_steps):
    
    ## Let's reshape arr in order to have array of dim: (n_seqs, ??)
    batch_total_size = n_seqs * n_steps
    n_batches = len(arr)//batch_total_size
    arr = arr[:(n_batches * batch_total_size)]
    arr_reshaped = arr.reshape((n_seqs, -1))
    for i in range(n_batches):
        x = arr_reshaped[:, i*n_steps:(i+1)*n_steps]
        y = np.concatenate([x[:, 1:], x[:,:1]], axis=1)
        yield x, y

In [26]:
gen_bashes = get_batches(swann_encoded, 10, 5)

In [27]:
a = next(gen_bashes)
a

(array([[42, 49, 32, 45, 47],
        [73, 59,  1, 69, 68],
        [59, 58,  1, 74, 69],
        [74, 55, 63, 68, 73],
        [68, 55, 74, 75, 72],
        [62, 59, 72,  1, 56],
        [59, 10,  1, 58, 59],
        [63, 67,  1, 74, 62],
        [72, 55, 74, 59,  1],
        [ 1, 77, 62, 63, 57]], dtype=int32), array([[49, 32, 45, 47, 42],
        [59,  1, 69, 68, 73],
        [58,  1, 74, 69, 59],
        [55, 63, 68, 73, 74],
        [55, 74, 75, 72, 68],
        [59, 72,  1, 56, 62],
        [10,  1, 58, 59, 59],
        [67,  1, 74, 62, 63],
        [55, 74, 59,  1, 72],
        [77, 62, 63, 57,  1]], dtype=int32))

In [28]:
def build_inputs(batch_size, num_steps):
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name="inputs")
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name="targets")
    keep_probabilites = tf.placeholder(tf.float32, name="keeps_probabilites")
    return inputs, targets, keep_probabilites

In [10]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    # Use a basic LSTM cell
    
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

In [11]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        x: Input tensor
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # That is, the shape should be batch_size*num_steps rows by lstm_size columns
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

In [12]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per batch_size per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

In [13]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [14]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [15]:
batch_size = 100        # Sequences per batch
num_steps = 100         # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001   # Learning rate
keep_prob = 0.5         # Dropout keep probability

In [33]:
epochs = 20
# Save every N iterations
save_every_n = 200

model = CharRNN(len(letters), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(swann_encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/20...  Training Step: 1...  Training loss: 4.5435...  6.2273 sec/batch
Epoch: 1/20...  Training Step: 2...  Training loss: 4.4422...  5.3930 sec/batch
Epoch: 1/20...  Training Step: 3...  Training loss: 3.8941...  5.3907 sec/batch
Epoch: 1/20...  Training Step: 4...  Training loss: 5.5439...  5.6176 sec/batch
Epoch: 1/20...  Training Step: 5...  Training loss: 4.0704...  7.0604 sec/batch
Epoch: 1/20...  Training Step: 6...  Training loss: 3.8400...  7.1247 sec/batch
Epoch: 1/20...  Training Step: 7...  Training loss: 3.6665...  6.7699 sec/batch
Epoch: 1/20...  Training Step: 8...  Training loss: 3.5008...  6.5596 sec/batch
Epoch: 1/20...  Training Step: 9...  Training loss: 3.3747...  5.3637 sec/batch
Epoch: 1/20...  Training Step: 10...  Training loss: 3.3890...  6.0869 sec/batch
Epoch: 1/20...  Training Step: 11...  Training loss: 3.3675...  6.2920 sec/batch
Epoch: 1/20...  Training Step: 12...  Training loss: 3.3394...  8.3177 sec/batch
Epoch: 1/20...  Training Step: 13... 

Epoch: 1/20...  Training Step: 103...  Training loss: 3.0658...  6.1920 sec/batch
Epoch: 1/20...  Training Step: 104...  Training loss: 3.0540...  6.3711 sec/batch
Epoch: 1/20...  Training Step: 105...  Training loss: 3.0568...  6.0572 sec/batch
Epoch: 1/20...  Training Step: 106...  Training loss: 3.0303...  6.0879 sec/batch
Epoch: 1/20...  Training Step: 107...  Training loss: 3.0425...  5.9323 sec/batch
Epoch: 1/20...  Training Step: 108...  Training loss: 3.0618...  5.9809 sec/batch
Epoch: 1/20...  Training Step: 109...  Training loss: 3.0414...  6.0774 sec/batch
Epoch: 1/20...  Training Step: 110...  Training loss: 3.0491...  5.9650 sec/batch
Epoch: 2/20...  Training Step: 111...  Training loss: 3.0821...  6.1608 sec/batch
Epoch: 2/20...  Training Step: 112...  Training loss: 3.0204...  6.1439 sec/batch
Epoch: 2/20...  Training Step: 113...  Training loss: 3.0206...  6.0023 sec/batch
Epoch: 2/20...  Training Step: 114...  Training loss: 3.0307...  6.1303 sec/batch
Epoch: 2/20...  

Epoch: 2/20...  Training Step: 203...  Training loss: 2.5562...  5.7608 sec/batch
Epoch: 2/20...  Training Step: 204...  Training loss: 2.5586...  5.5784 sec/batch
Epoch: 2/20...  Training Step: 205...  Training loss: 2.5813...  6.6674 sec/batch
Epoch: 2/20...  Training Step: 206...  Training loss: 2.5602...  6.9952 sec/batch
Epoch: 2/20...  Training Step: 207...  Training loss: 2.5337...  5.4921 sec/batch
Epoch: 2/20...  Training Step: 208...  Training loss: 2.5256...  5.7041 sec/batch
Epoch: 2/20...  Training Step: 209...  Training loss: 2.5576...  5.5678 sec/batch
Epoch: 2/20...  Training Step: 210...  Training loss: 2.5617...  5.7648 sec/batch
Epoch: 2/20...  Training Step: 211...  Training loss: 2.5360...  5.6968 sec/batch
Epoch: 2/20...  Training Step: 212...  Training loss: 2.5457...  5.6860 sec/batch
Epoch: 2/20...  Training Step: 213...  Training loss: 2.5359...  5.7742 sec/batch
Epoch: 2/20...  Training Step: 214...  Training loss: 2.5367...  5.4979 sec/batch
Epoch: 2/20...  

Epoch: 3/20...  Training Step: 303...  Training loss: 2.3413...  5.4654 sec/batch
Epoch: 3/20...  Training Step: 304...  Training loss: 2.3530...  5.9798 sec/batch
Epoch: 3/20...  Training Step: 305...  Training loss: 2.3318...  5.5209 sec/batch
Epoch: 3/20...  Training Step: 306...  Training loss: 2.3372...  5.8917 sec/batch
Epoch: 3/20...  Training Step: 307...  Training loss: 2.3578...  5.9417 sec/batch
Epoch: 3/20...  Training Step: 308...  Training loss: 2.3177...  5.4068 sec/batch
Epoch: 3/20...  Training Step: 309...  Training loss: 2.3504...  5.5013 sec/batch
Epoch: 3/20...  Training Step: 310...  Training loss: 2.3460...  6.7357 sec/batch
Epoch: 3/20...  Training Step: 311...  Training loss: 2.3415...  5.5312 sec/batch
Epoch: 3/20...  Training Step: 312...  Training loss: 2.3268...  5.9671 sec/batch
Epoch: 3/20...  Training Step: 313...  Training loss: 2.3135...  5.4254 sec/batch
Epoch: 3/20...  Training Step: 314...  Training loss: 2.3024...  5.3522 sec/batch
Epoch: 3/20...  

Epoch: 4/20...  Training Step: 403...  Training loss: 2.1845...  5.4964 sec/batch
Epoch: 4/20...  Training Step: 404...  Training loss: 2.1663...  5.3514 sec/batch
Epoch: 4/20...  Training Step: 405...  Training loss: 2.2000...  5.5324 sec/batch
Epoch: 4/20...  Training Step: 406...  Training loss: 2.1890...  5.7425 sec/batch
Epoch: 4/20...  Training Step: 407...  Training loss: 2.1832...  5.8111 sec/batch
Epoch: 4/20...  Training Step: 408...  Training loss: 2.1596...  5.6621 sec/batch
Epoch: 4/20...  Training Step: 409...  Training loss: 2.1919...  5.7347 sec/batch
Epoch: 4/20...  Training Step: 410...  Training loss: 2.1877...  5.8595 sec/batch
Epoch: 4/20...  Training Step: 411...  Training loss: 2.1782...  5.5172 sec/batch
Epoch: 4/20...  Training Step: 412...  Training loss: 2.2101...  5.5447 sec/batch
Epoch: 4/20...  Training Step: 413...  Training loss: 2.1701...  5.5436 sec/batch
Epoch: 4/20...  Training Step: 414...  Training loss: 2.1971...  5.2989 sec/batch
Epoch: 4/20...  

...

Epoch: 17/20...  Training Step: 1787...  Training loss: 1.5132...  5.3709 sec/batch
Epoch: 17/20...  Training Step: 1788...  Training loss: 1.5029...  5.4398 sec/batch
Epoch: 17/20...  Training Step: 1789...  Training loss: 1.5332...  5.3926 sec/batch
Epoch: 17/20...  Training Step: 1790...  Training loss: 1.5187...  5.4282 sec/batch
Epoch: 17/20...  Training Step: 1791...  Training loss: 1.4928...  5.3320 sec/batch
Epoch: 17/20...  Training Step: 1792...  Training loss: 1.5227...  5.4705 sec/batch
Epoch: 17/20...  Training Step: 1793...  Training loss: 1.5131...  5.3406 sec/batch
Epoch: 17/20...  Training Step: 1794...  Training loss: 1.5248...  5.4465 sec/batch
Epoch: 17/20...  Training Step: 1795...  Training loss: 1.5007...  5.4343 sec/batch
Epoch: 17/20...  Training Step: 1796...  Training loss: 1.5297...  5.3820 sec/batch
Epoch: 17/20...  Training Step: 1797...  Training loss: 1.4634...  5.4291 sec/batch
Epoch: 17/20...  Training Step: 1798...  Training loss: 1.4783...  5.3957 se

Epoch: 18/20...  Training Step: 1885...  Training loss: 1.4985...  5.4131 sec/batch
Epoch: 18/20...  Training Step: 1886...  Training loss: 1.4659...  5.4158 sec/batch
Epoch: 18/20...  Training Step: 1887...  Training loss: 1.4685...  5.4735 sec/batch
Epoch: 18/20...  Training Step: 1888...  Training loss: 1.4680...  5.3694 sec/batch
Epoch: 18/20...  Training Step: 1889...  Training loss: 1.4747...  5.6927 sec/batch
Epoch: 18/20...  Training Step: 1890...  Training loss: 1.4636...  5.3795 sec/batch
Epoch: 18/20...  Training Step: 1891...  Training loss: 1.4872...  5.4394 sec/batch
Epoch: 18/20...  Training Step: 1892...  Training loss: 1.5011...  5.4287 sec/batch
Epoch: 18/20...  Training Step: 1893...  Training loss: 1.4920...  5.5439 sec/batch
Epoch: 18/20...  Training Step: 1894...  Training loss: 1.5030...  5.3983 sec/batch
Epoch: 18/20...  Training Step: 1895...  Training loss: 1.5174...  5.4552 sec/batch
Epoch: 18/20...  Training Step: 1896...  Training loss: 1.4902...  5.5380 se

Epoch: 19/20...  Training Step: 1983...  Training loss: 1.4546...  5.6472 sec/batch
Epoch: 19/20...  Training Step: 1984...  Training loss: 1.4666...  5.4679 sec/batch
Epoch: 19/20...  Training Step: 1985...  Training loss: 1.4776...  5.5258 sec/batch
Epoch: 19/20...  Training Step: 1986...  Training loss: 1.4745...  5.3167 sec/batch
Epoch: 19/20...  Training Step: 1987...  Training loss: 1.4506...  5.3376 sec/batch
Epoch: 19/20...  Training Step: 1988...  Training loss: 1.4486...  5.3455 sec/batch
Epoch: 19/20...  Training Step: 1989...  Training loss: 1.4298...  5.4260 sec/batch
Epoch: 19/20...  Training Step: 1990...  Training loss: 1.4383...  5.3766 sec/batch
Epoch: 19/20...  Training Step: 1991...  Training loss: 1.4930...  5.4551 sec/batch
Epoch: 19/20...  Training Step: 1992...  Training loss: 1.4717...  5.3219 sec/batch
Epoch: 19/20...  Training Step: 1993...  Training loss: 1.4639...  5.3952 sec/batch
Epoch: 19/20...  Training Step: 1994...  Training loss: 1.4680...  5.4140 se

Epoch: 19/20...  Training Step: 2081...  Training loss: 1.4323...  5.4536 sec/batch
Epoch: 19/20...  Training Step: 2082...  Training loss: 1.4579...  5.3482 sec/batch
Epoch: 19/20...  Training Step: 2083...  Training loss: 1.4527...  5.4190 sec/batch
Epoch: 19/20...  Training Step: 2084...  Training loss: 1.4567...  5.2977 sec/batch
Epoch: 19/20...  Training Step: 2085...  Training loss: 1.4284...  5.4179 sec/batch
Epoch: 19/20...  Training Step: 2086...  Training loss: 1.4313...  5.5376 sec/batch
Epoch: 19/20...  Training Step: 2087...  Training loss: 1.4395...  5.5112 sec/batch
Epoch: 19/20...  Training Step: 2088...  Training loss: 1.4436...  5.2720 sec/batch
Epoch: 19/20...  Training Step: 2089...  Training loss: 1.4469...  5.5080 sec/batch
Epoch: 19/20...  Training Step: 2090...  Training loss: 1.4610...  5.3589 sec/batch
Epoch: 20/20...  Training Step: 2091...  Training loss: 1.4897...  5.4125 sec/batch
Epoch: 20/20...  Training Step: 2092...  Training loss: 1.4291...  5.3610 se

Epoch: 20/20...  Training Step: 2179...  Training loss: 1.4454...  5.5149 sec/batch
Epoch: 20/20...  Training Step: 2180...  Training loss: 1.4580...  5.3406 sec/batch
Epoch: 20/20...  Training Step: 2181...  Training loss: 1.4436...  5.3999 sec/batch
Epoch: 20/20...  Training Step: 2182...  Training loss: 1.4447...  5.5039 sec/batch
Epoch: 20/20...  Training Step: 2183...  Training loss: 1.4214...  5.3653 sec/batch
Epoch: 20/20...  Training Step: 2184...  Training loss: 1.4146...  5.4668 sec/batch
Epoch: 20/20...  Training Step: 2185...  Training loss: 1.4704...  5.2516 sec/batch
Epoch: 20/20...  Training Step: 2186...  Training loss: 1.4125...  5.3221 sec/batch
Epoch: 20/20...  Training Step: 2187...  Training loss: 1.4116...  5.4098 sec/batch
Epoch: 20/20...  Training Step: 2188...  Training loss: 1.4309...  5.5369 sec/batch
Epoch: 20/20...  Training Step: 2189...  Training loss: 1.4332...  5.3056 sec/batch
Epoch: 20/20...  Training Step: 2190...  Training loss: 1.4566...  5.3116 se

In [39]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [46]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(letters), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = letter2int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(letters))
        samples.append(int2letters[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(letters))
            samples.append(int2letters[c])
        
    return ''.join(samples)

In [47]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i2200_l512.ckpt'

In [48]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(letters), prime="Far")
print(samp)

Farly, bothing
that I made it it will be seen to my mother the streem of the service of
the soming stairs, on whom he had conceived that it woush in his still
shaded, simply to him also. She was all the care whom I was to have been to time, with
an insistance than they were seemed to see him, and such see her, seemed only
the same mare to the corring of the fiest than a fine common would be, and who
asked him as there, she had asked, he could be allowed that the crate
of women of the fellow of her contronic crosming whose painte should see her to
mind have couree once to him with which they were stoped to have been her, was not a least
interpretence. I see that these way to have seed himself that I would say that I had
allowed him was not been able to, in a story was in him whom she was the same
time of head the soll more of a care of secret of his sorn whom I had
not say a life whom he had stained as secies of the carraties. His strength, with what
had been indued the sun to-the frien

In [49]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(letters), prime="Far")
print(samp)

Farnd teree tor one sisd, thit an wha ne he sher to an tat is ore tho s tou thar serit onte hhes af tout hh to hard tinse tin ha the ate hh er te ar tos wont ha tore wor on ored it is tond
whe he ade hes winn onr
os ar thi h he
to th onr onet, or theminn
tin thin se tor an sot herir ho t oas an ot are hi n on ale and int aris ons he thos and an to tore the as tora ware he win ont teese won hh amins tor
so an he se wanlde as whe hh ase sare are hin os wand tare hes tate the antin on the wou hh as whom he the he te th in tis sert ane th arter ifd of th and on or he te ther thin hee whhas thir os and thin tast an the hed
sen ito he hos ta sh tis
tar se hhe whe ser tee thit whe the the an ho wentat ate the whe so s or al oned on here whir hos werase sirised at oun are tot he whe tiresirtin sit othe the to ar te thi the ansint as at it an har ar ant th to sathe se won the as and arer ha thind ooreren an in intint in hinth wim os ote he at hee tare wis o the wos athe the whe wanl ad
toned as

In [50]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(letters), prime="Far")
print(samp)

Farre sond the piretto thate the cone wate a somiry tome that to he had, some and her to to mattrer that
had his
seadion, as thims ontion, ass the wers with an he sunce of her sere and thing the minters to as ast antelious alloted, to sto mark to sore, thourd an ortally, at that imen in a cerse of thoughe, shome
welt say of to mind, when I she deven tine though that a ceall of that, he chasis of
her
soress, it an theight which when
storder, and ald that the
cortision,
a distind and to me whinger a ferticion this sertaring tho take, hiss, the sulind on to the chart, whe that
hers wat treartile, the chonce to hem sore, in the pillion, thas thiss ald treaks a ponting the
mont in at a mingrasing. And in who herser and
an those astelliched to a sithing as of hid sorcious if as and ofeer of her serendst which,
a stillly, at her sume to
mering, that the poothous whan as himed beene though him seem, then
himsencith anderted, all a pastoun of, his sounghe with stringed of the mont, in it sees, 

In [51]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(letters), prime="Far")
print(samp)

Farchating when he had night
a state the partir to him any heart, had beauted himself a peeple alothing, she would have treing himsing a creath to soce to the persents when he had astained to making her
to so some imeation of
seeping him one of the sone of the forter would not, as a strants in a cass and would have been atchence of his mas or which shisherse to teese or and to tings that the pleasure in the countrose, and seemon with any more a forct of tering at the porter of a sompthan, at whech, in the sain of this woman and so when the paises which the sace of mement, then theyss, houres which he seed alone.
At her an experion to her have alone of those
stations and trenser, and had not and trows
a sender only the world, and sometimes of treating of sumpitiand take him and the same, at the peris of his pasticult and who to hard, as heart and sore at the wind then thing that his mand on the sample,
side of the seeper in the concear to have theme, a moment of myself-all the crathing 